In [17]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import random
from icecream import ic
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
from itertools import accumulate

In [18]:
logging.basicConfig(level=logging.DEBUG)

## Nearest Neighbor (Greedy)

In [19]:
def nearest_neighbor_tsp(start_city_index=0):
    """
    Risolve il TSP utilizzando l'algoritmo Greedy di Nearest Neighbor.
    :param start_city_index: Indice della città di partenza
    :return: Lista dell'ordine delle città visitate e lunghezza totale del percorso
    """
    dist_matrix=DIST_MATRIX.copy()
    visited = np.full(len(CITIES), False)  # Array per tenere traccia delle città visitate
    city = start_city_index  
    visited[city] = True

    tsp = [city]  # Lista per mantenere l'ordine delle città visitate

    # Loop fino a quando tutte le città non sono state visitate
    while not np.all(visited):
        # Imposta le distanze delle città già visitate a infinito per escluderle dalla ricerca del più vicino
        dist_matrix[:, city] = np.inf
        # Trova la città più vicina che non è ancora stata visitata
        closest_city = np.argmin(dist_matrix[city])
        # Marca la città trovata come visitata
        visited[closest_city] = True
        # Aggiorna l'ordine delle città visitate
        tsp.append(int(closest_city))
        # Aggiorna la città corrente
        city = closest_city

    # Ritorna alla città di partenza
    tsp.append(start_city_index)
    
    total_distance = tsp_cost(tsp,DIST_MATRIX)
    
    return tsp, total_distance

In [20]:
def valid(tsp):
    """
    Verifica la validità di un percorso TSP.
    """
    tsp = np.array(tsp)
    
    if tsp[0] != tsp[-1]:
        return False
    
    # Verifica che tutte le città siano visitate esattamente una volta
    cities_visited = tsp[:-1]
    expected_cities = np.arange(len(CITIES))
    
    return (len(np.unique(cities_visited)) == len(CITIES) and 
            np.all(np.isin(expected_cities, cities_visited)))

def tsp_cost(route, dist_matrix):
    """Calcola il costo del percorso e la sua validità."""
    route = np.array(route)
    # Calcola le distanze tra città consecutive
    total_distance = np.sum([dist_matrix[route[i], route[i+1]] 
                           for i in range(len(route)-1)])
    return (total_distance, valid(route))

## Inversion mutation

In [21]:
def tweak(tsp):
    """Esegue un'inversion mutation su una soluzione TSP"""
    tsp = np.array(tsp)
    # Seleziona due punti casuali nel percorso per eseguire l'inversione
    a, b = sorted(np.random.choice(range(1, len(tsp) - 1), 2, replace=False))
     # Inverte il sotto-percorso tra i due punti selezionati
    tsp[a:b+1] = np.flip(tsp[a:b+1])
    return tsp.tolist()


##  Simulated Annealing (SA) by Initial Population.

In [22]:
def simulated_annealing_population(dist_matrix, initial_temp, cooling_rate, stop_temp, population_size=100):
    """Algoritmo di Simulated Annealing per generare una popolazione iniziale"""
    dist_matrix = np.array(dist_matrix)
    # Trova una soluzione iniziale con il metodo Greedy
    current_solution, current_cost = nearest_neighbor_tsp(start_city_index=0)
    best_solution = current_solution[:]
    best_cost = current_cost

    population = [(best_solution, best_cost)]
    temperature = initial_temp

    while temperature > stop_temp:
        new_solution = tweak(current_solution[:])
        new_cost = tsp_cost(new_solution, dist_matrix)

        if (new_cost[0] < current_cost[0] or 
            np.random.random() < np.exp((current_cost[0] - new_cost[0]) / temperature)):
            current_solution = new_solution
            current_cost = new_cost

            if new_cost[0] < best_cost[0]:
                best_solution = new_solution
                best_cost = new_cost
    
            population.append((current_solution, current_cost))
            
        temperature *= cooling_rate
    # Ordina la popolazione per costo e restituisci i migliori 'population_size' individui    
    population = np.array(population, dtype=object)
    sorted_indices = np.argsort([cost[0] for _, cost in population])
    population = population[sorted_indices]
    
    return population[:population_size].tolist()

In [23]:
def initial_population(pop_size):
    """Inizializza la popolazione usando parametri predefiniti."""
    initial_temp = 1000
    cooling_rate = 0.992
    stop_temp = 1e-200
    # Genera la popolazione iniziale con Simulated Annealing
    return simulated_annealing_population(DIST_MATRIX, initial_temp,
                                        cooling_rate, stop_temp, pop_size)


##  Crossover Inver-Over

In [24]:
def inver_over_crossover(parent1, parent2, num_iterations=1):
    """Applica il crossover inver-over """
    parent1, parent2 = np.array(parent1), np.array(parent2)
    child = parent1.copy()
 # Esegui una serie di iterazioni di crossover per creare il figlio
    for _ in range(num_iterations):
        start_index = np.random.randint(1, len(child) - 2)
        start_city = child[start_index]
         # Seleziona casualmente una città di destinazione da uno dei genitori
        if np.random.random() < 0.5:
            valid_cities = child[1:-1][child[1:-1] != start_city]
            end_city = np.random.choice(valid_cities)
        else:
            valid_cities = parent2[1:-1][parent2[1:-1] != start_city]
            end_city = np.random.choice(valid_cities)
        # Ottieni le posizioni della città di partenza e di destinazione nel figlio
        start_pos = np.where(child == start_city)[0][0]
        end_pos = np.where(child == end_city)[0][0]
        
        if start_pos > end_pos:
            start_pos, end_pos = end_pos, start_pos
          # Inverti il sotto-percorso tra le due posizioni
        child[start_pos:end_pos + 1] = np.flip(child[start_pos:end_pos + 1])
    # Assicurati che il figlio termini nella stessa città di partenza
    if child[0] != child[-1]:
        child = np.append(child, child[0])
    
    return child.tolist()

In [25]:
def tournament_selection(population, k=5):
    """Selezione torneo usando NumPy."""
     # Seleziona casualmente 'k' individui dalla popolazione
    selected_indices = np.random.choice(len(population), k, replace=False)
    selected = np.array(population, dtype=object)[selected_indices]
     # Ritorna l'individuo con il costo minore tra quelli selezionati
    return min(selected, key=lambda x: x[1][0])  # x[1][0] accede al costo


## Inversion mutation

In [26]:
def inversion_mutation(tsp):
    """Esegue un'inversion mutation """
    tsp = np.array(tsp)
    a, b = sorted(np.random.choice(range(1, len(tsp) - 1), 2, replace=False))
    tsp[a:b+1] = np.flip(tsp[a:b+1])
    return tsp.tolist()


## EA with adaptive mutation and multiple restart with simulated anneling.

In [27]:


def genetic_algorithm_tsp_adaptive_mutation(
    
    dist_matrix,
    population_size=100, 
    tournament_size=10, 
    num_generations=10000, 
    initial_mutation_rate=0.1, 
    crossover_iterations=1,
    max_generations_without_improvement=500,
    mutation_increase_factor=1.5,
    mutation_decrease_factor=0.7,
    mutation_threshold=1.3,  # Soglia per il riavvio
    initial_temp=1000,      
    cooling_rate=0.992,
    stop_temp=1e-200,

):
    """Risolve il TSP utilizzando un Algoritmo Genetico con mutazione adattiva."""
    dist_matrix = np.array(dist_matrix)
    population = initial_population(population_size)
    current_mutation_rate = initial_mutation_rate

    best_solution, best_cost = min(population, key=lambda x: x[1][0])
    all_time_best_solution = best_solution
    all_time_best_cost = best_cost
    generations_without_improvement = 0
    

    for generation in tqdm(range(num_generations)):
        # Controlla se è necessario riavviare con SA
        if current_mutation_rate > mutation_threshold:
            print(f"\nMutation rate ({current_mutation_rate:.3f}) exceeded threshold ({mutation_threshold})")
            print("Restarting with Simulated Annealing...")
            
            # Esegui SA e ottieni nuova popolazione
            new_population = simulated_annealing_population(
                dist_matrix, 
                initial_temp, 
                cooling_rate, 
                stop_temp, 
                population_size
            )
            population = sorted(new_population, key=lambda x: x[1][0])[:population_size]
            # Reset del tasso di mutazione
            current_mutation_rate = initial_mutation_rate
            print("Population refreshed and mutation rate reset")

        new_population = []

        while len(new_population) < population_size:
            parent1 = tournament_selection(population, tournament_size)
            parent2 = tournament_selection(population, tournament_size)
            
            child = inver_over_crossover(parent1[0], parent2[0], crossover_iterations)
            
            if np.random.random() < current_mutation_rate:
                child = inversion_mutation(child)

            child_cost = tsp_cost(child, dist_matrix)
            new_population.append((child, child_cost))

        combined_population = np.array(new_population + population, dtype=object)
        sorted_indices = np.argsort([cost[0] for _, cost in combined_population])
        population = combined_population[sorted_indices][:population_size].tolist()

        new_best_solution, new_best_cost = min(population, key=lambda x: x[1][0])

        # Aggiorna il miglior risultato di sempre se necessario
        if new_best_cost[0] < all_time_best_cost[0]:
            all_time_best_solution = new_best_solution
            all_time_best_cost = new_best_cost

        if new_best_cost[0] < best_cost[0]:
            best_solution, best_cost = new_best_solution, new_best_cost
            generations_without_improvement = 0
            current_mutation_rate = max(initial_mutation_rate, 
                                     current_mutation_rate * mutation_decrease_factor)
        else:
            generations_without_improvement += 1

        if generations_without_improvement > max_generations_without_improvement:
            current_mutation_rate *= mutation_increase_factor
            generations_without_improvement = 0

        if generation % 1000 == 0:
            print(f"Generazione {generation}: "
                  f"Costo migliore corrente = {best_cost[0]:.2f} | "
                  f"Costo migliore assoluto = {all_time_best_cost[0]:.2f} | "
                  f"Tasso di mutazione = {current_mutation_rate:.3f} | "
                  f"Valido = {best_cost[1]}")
    
    return all_time_best_cost


## Vanuatu

In [28]:

CITIES = pd.read_csv('./cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
genetic_algorithm_tsp_adaptive_mutation(
    DIST_MATRIX,
    num_generations=0, 
)



0it [00:00, ?it/s]


(np.float64(1345.5449564733112), np.True_)

For vanuatu just applying simulated anneling to get the best solution is enough

## Italy

In [29]:

CITIES = pd.read_csv('./cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
genetic_algorithm_tsp_adaptive_mutation(
    DIST_MATRIX,
    mutation_increase_factor=1.3,
    mutation_decrease_factor=0.7
)

  0%|          | 8/10000 [00:00<02:22, 70.15it/s]

Generazione 0: Costo migliore corrente = 4436.03 | Costo migliore assoluto = 4436.03 | Tasso di mutazione = 0.100 | Valido = True


 10%|█         | 1011/10000 [00:14<02:04, 72.45it/s]

Generazione 1000: Costo migliore corrente = 4258.07 | Costo migliore assoluto = 4258.07 | Tasso di mutazione = 0.130 | Valido = True


 20%|██        | 2009/10000 [00:28<01:53, 70.68it/s]

...

Generazione 8000: Costo migliore corrente = 4172.76 | Costo migliore assoluto = 4172.76 | Tasso di mutazione = 0.286 | Valido = True


 90%|█████████ | 9009/10000 [02:26<00:19, 50.09it/s]

Generazione 9000: Costo migliore corrente = 4172.76 | Costo migliore assoluto = 4172.76 | Tasso di mutazione = 0.483 | Valido = True


100%|██████████| 10000/10000 [02:45<00:00, 60.40it/s]


(np.float64(4172.7626139164095), np.True_)

## Russia

In [30]:

CITIES = pd.read_csv('./cities/russia.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
genetic_algorithm_tsp_adaptive_mutation(DIST_MATRIX)

  0%|          | 9/10000 [00:00<03:39, 45.53it/s]

Generazione 0: Costo migliore corrente = 34870.45 | Costo migliore assoluto = 34870.45 | Tasso di mutazione = 0.100 | Valido = True


 10%|█         | 1005/10000 [00:22<02:48, 53.24it/s]

Generazione 1000: Costo migliore corrente = 33564.66 | Costo migliore assoluto = 33564.66 | Tasso di mutazione = 0.100 | Valido = True


 20%|██        | 2007/10000 [00:41<02:31, 52.76it/s]

...


Mutation rate (1.709) exceeded threshold (1.3)
Restarting with Simulated Annealing...


 99%|█████████▊| 9854/10000 [03:45<00:46,  3.11it/s]

Population refreshed and mutation rate reset


100%|██████████| 10000/10000 [03:49<00:00, 43.58it/s]


(np.float64(33441.17816395336), np.True_)

## Us

In [31]:

CITIES = pd.read_csv('./cities/us.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
genetic_algorithm_tsp_adaptive_mutation(DIST_MATRIX)

  0%|          | 5/10000 [00:00<04:05, 40.70it/s]

Generazione 0: Costo migliore corrente = 43226.50 | Costo migliore assoluto = 43226.50 | Tasso di mutazione = 0.100 | Valido = True


 10%|█         | 1002/10000 [00:31<04:20, 34.53it/s]

Generazione 1000: Costo migliore corrente = 41706.29 | Costo migliore assoluto = 41706.29 | Tasso di mutazione = 0.100 | Valido = True


 20%|██        | 2006/10000 [01:07<04:09, 32.08it/s]

...

Generazione 8000: Costo migliore corrente = 39891.44 | Costo migliore assoluto = 39891.44 | Tasso di mutazione = 0.100 | Valido = True


 90%|█████████ | 9006/10000 [05:02<00:32, 30.77it/s]

Generazione 9000: Costo migliore corrente = 39871.46 | Costo migliore assoluto = 39871.46 | Tasso di mutazione = 0.100 | Valido = True


100%|██████████| 10000/10000 [05:34<00:00, 29.89it/s]


(np.float64(39858.250617746125), np.True_)

## China

In [32]:

CITIES = pd.read_csv('./cities/china.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
genetic_algorithm_tsp_adaptive_mutation(
    DIST_MATRIX,
    num_generations=20000, 
)

  0%|          | 3/20000 [00:00<16:48, 19.82it/s]

Generazione 0: Costo migliore corrente = 63962.92 | Costo migliore assoluto = 63962.92 | Tasso di mutazione = 0.100 | Valido = True


  5%|▌         | 1004/20000 [00:52<15:23, 20.58it/s]

Generazione 1000: Costo migliore corrente = 61845.85 | Costo migliore assoluto = 61845.85 | Tasso di mutazione = 0.100 | Valido = True


 10%|█         | 2003/20000 [01:42<14:55, 20.11it/s]

...

Generazione 18000: Costo migliore corrente = 54243.14 | Costo migliore assoluto = 54243.14 | Tasso di mutazione = 0.100 | Valido = True


 95%|█████████▌| 19003/20000 [16:53<00:50, 19.78it/s]

Generazione 19000: Costo migliore corrente = 54161.25 | Costo migliore assoluto = 54161.25 | Tasso di mutazione = 0.100 | Valido = True


100%|██████████| 20000/20000 [17:39<00:00, 18.88it/s]


(np.float64(54028.37760265771), np.True_)